In [2]:
import os, sys, io, random
import string
import numpy as np
import pandas as pd
from importlib import reload

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

from IPython.display import HTML

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, CustomJS, Slider, HoverTool, NumeralTickFormatter, Arrow, NormalHead
from bokeh.models import LinearAxis, Range1d, DataRange1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot, column
import panel as pn
import panel.widgets as pnw
pn.extension()
from pybioviz import dashboards, utils, plotters

Original genome_features_veiwer to modfify

In [3]:

app = dashboards.genome_features_viewer('example.gff')
app

Column(margin=20, sizing_mode='stretch_width', width_policy='max')
    [0] Row
        [0] TextInput(name='location', width=150)
        [1] IntSlider(end=500, name='zoom', start=1, step=5, value=100, width=100)
        [2] TextInput(name='find_gene', width=220)
        [3] Select(name='chrom', width=220)
        [4] Button(name='<', width=40)
        [5] Button(name='>', width=40)
    [1] IntSlider(end=8514, name='start', step=500, value=1, width=900)
    [2] Column(sizing_mode='stretch_width')
        [0] Bokeh(Figure, height=100, margin=10)
        [1] Bokeh(None, height=50, margin=10)

## Create a quick helper to extract the seq from gb files

In [4]:
def genbank_to_sequence(gb_file, key=0):
    """Read genbank record features"""

    if gb_file is None or not os.path.exists(gb_file):
        return
    rec = list(SeqIO.parse(open(gb_file,'r'),'genbank'))[key]
    return rec.seq

# Primary plasmid feature view function

In [7]:
def plasmid_features_viewer(gb_file, plot_width=900):
    """Gene feature viewer app"""
    
    if gb_file is None:
        return
    
    features = utils.genbank_to_features(gb_file)
    
    loc_pane = pnw.TextInput(name='location',value='',width=150)
    search_pane = pnw.TextInput(name='find_gene',value='',width=220)
    slider = pnw.IntSlider(name='start',start=0,end=10000,step=500,value=1,width=plot_width)
    xzoom_slider = pnw.IntSlider(name='zoom',start=1,end=500,value=100,step=5,width=100)
    left_button = pnw.Button(name='<',width=40)
    right_button = pnw.Button(name='>',width=40)
    fasta_seq = genbank_to_sequence(gb_file)
    feature_pane = pn.pane.Bokeh(height=100,margin=10)
    seq_pane = pn.pane.Bokeh(height=50, margin = 10)
    debug_pane = pn.pane.Str('debug',width=200,style={'background':'yellow','margin': '4pt'})
    
    seqlen = len(fasta_seq)
    slider.end = seqlen
    
    def search_features(event):
        """Find a feature"""
        
        term = search_pane.value        
        feats = utils.genbank_to_features(gb_file)
        df = utils.features_to_dataframe(feats)    
        df['gene'] = df.gene.fillna('')
        f = df[df.gene.str.contains(term)].iloc[0]
        #debug_pane.object = str(f.start)
        slider.value = int(f.start)-100
        update(event)
        return   
    
    def update(event):      
        print (event.obj.name)
        if event.obj.name in ['start', 'zoom']:
            xzoom = xzoom_slider.value*200
            start = int(slider.value)
            N = xzoom/2
            end = int(start+N)
            loc_pane.value = str(start)+':'+str(end)            
        elif event.obj.name == 'location':            
            vals = loc_pane.value.split(':')
            start = int(vals[0])
            end = int(vals[1])
            slider.value = start        
            

        p = feature_pane.object
        p.x_range.start = start
        p.x_range.end = end
    
        sequence = fasta_seq[start: end]
        seq_pane.object = plotters.plot_sequence(sequence, plot_width, plot_height=50,fontsize='9pt',xaxis=False)            

        
    slider.param.watch(update,'value',onlychanged=True)
  
    xzoom_slider.param.watch(update,'value')       
    search_pane.param.watch(search_features,'value')    
    loc_pane.param.watch(update,'value',onlychanged=True)    

    feature_pane.object = plotters.plot_features(features, 0, 10000, plot_width=plot_width, tools="", rows=4)
    seq_pane.object = plotters.plot_sequence(fasta_seq, plot_width, plot_height=50,fontsize='9pt',xaxis=False)  
    top = pn.Row(loc_pane,xzoom_slider)
    main = pn.Column(feature_pane, seq_pane, sizing_mode='stretch_width')
    app = pn.Column(top,slider,main, sizing_mode='stretch_width',width_policy='max',margin=20)
    return app

In [8]:
# Test panel for plasmid features viewer

plasmid_features_viewer('d378_attb-entry.gb')




Column(margin=20, sizing_mode='stretch_width', width_policy='max')
    [0] Row
        [0] TextInput(name='location', width=150)
        [1] IntSlider(end=500, name='zoom', start=1, step=5, value=100, width=100)
    [1] IntSlider(end=4411, name='start', step=500, value=1, width=900)
    [2] Column(sizing_mode='stretch_width')
        [0] Bokeh(Figure, height=100, margin=10)
        [1] Bokeh(Figure, height=50, margin=10)

In [45]:
seq_pane = pn.pane.Bokeh(height=100,margin=10)
x= genbank_to_sequence("d378_attb-entry.gb")
seq_pane.object = plotters.plot_sequence(x[1:100])
main = pn.Column(seq_pane, sizing_mode='stretch_width')
main


Column(sizing_mode='stretch_width')
    [0] Bokeh(Figure, height=100, margin=10)

In [22]:
import bokeh.models.widgets as bhw
from bokeh.models.callbacks import CustomJS

loc = pnw.TextInput(name='location',value='',width=180)
s=pnw.IntSlider(name='value',start=0,end=100,value=20)

p = plotters.test2(plot_width=800)
pp = pn.panel(p)

def callback(target, event):
    print (event)
    
pp.link(s, callbacks={'object': callback}) 

jsupdateplot = '''
    r = target.x_range.end - target.x_range.start
    target.x_range.start = source.value
    target.x_range.end = source.value+r
'''
s.jslink(p, code={'value': jsupdateplot}) #value='x_range.start')

jsupdateloc = '''   
    if (source.start>0){
       target.value = Math.round(source.start)+':'+Math.round(source.end)
    }
'''


pn.Column(loc,s, pp)

Column
    [0] TextInput(name='location', width=180)
    [1] IntSlider(end=100, name='value', value=20)
    [2] Bokeh(Figure)

In [84]:
def genome_features_viewer(gb_file, plot_width=900):
    """Gene feature viewer app"""
    
    if gb_file is None:
        return
    
    features = utils.genbank_to_features(gb_file)
    df = utils.features_to_dataframe(features)
    
    loc_pane = pnw.TextInput(name='location',value='',width=150)
    search_pane = pnw.TextInput(name='find_gene',value='',width=220)
    slider = pnw.IntSlider(name='start',start=0,end=10000,step=500,value=1,width=plot_width)
    xzoom_slider = pnw.IntSlider(name='zoom',start=1,end=500,value=100,step=5,width=100)
    left_button = pnw.Button(name='<',width=40)
    right_button = pnw.Button(name='>',width=40)
    fasta_seq = genbank_to_sequence(gb_file)
    feature_pane = pn.pane.Bokeh(height=100,margin=10)
    seq_pane = pn.pane.Bokeh(height=50, margin = 10)
    debug_pane = pn.pane.Str('debug',width=200,style={'background':'yellow','margin': '4pt'})
    
    seqlen = len(fasta_seq)
    slider.end = seqlen
    
    def search_features(event):
        """Find a feature"""
        
        term = search_pane.value        
        feats = utils.genbank_to_features(gb_file)
        df = utils.features_to_dataframe(feats)    
        df['gene'] = df.gene.fillna('')
        f = df[df.gene.str.contains(term)].iloc[0]
        #debug_pane.object = str(f.start)
        slider.value = int(f.start)-100
        update(event)
        return   
    
    def pan(event):
        p = feature_pane.object
        rng = p.x_range.end-p.x_range.start        
        inc = int(rng/10)
        print (event.obj.name)
        if event.obj.name == '<':
            slider.value = int(slider.value) - inc        
        else:
            slider.value = int(slider.value) + inc   
        update(event)
        return
    
    def update(event):      
        print (event.obj.name)
        if event.obj.name in ['start', 'zoom']:
            xzoom = xzoom_slider.value*200
            start = int(slider.value)
            N = xzoom/2
            end = int(start+N)
            loc_pane.value = str(start)+':'+str(end)            
        elif event.obj.name == 'location':            
            vals = loc_pane.value.split(':')
            start = int(vals[0])
            end = int(vals[1])
            slider.value = start        
            

        p = feature_pane.object
        p.x_range.start = start
        p.x_range.end = end
    
        sequence = fasta_seq[start: end]
        seq_pane.object = plotters.plot_sequence(sequence, plot_width, plot_height=50,fontsize='9pt',xaxis=False)            

        
    slider.param.watch(update,'value',onlychanged=True)
    #slider.param.trigger('value')    
    xzoom_slider.param.watch(update,'value')       
    search_pane.param.watch(search_features,'value')    
    loc_pane.param.watch(update,'value',onlychanged=True)    
    left_button.param.watch(pan,'clicks')
    right_button.param.watch(pan,'clicks')
    #debug_pane.object = utils.get_fasta_names(ref_file)[0] 


    feature_pane.object = plotters.plot_features(features, 0, 10000, plot_width=plot_width, tools="", rows=4)
    seq_pane.object = plotters.plot_sequence(fasta_seq, plot_width, plot_height=50,fontsize='9pt',xaxis=False)  
    #side = pn.Column(file_input,css_classes=['form'],width=200,margin=20)
    top = pn.Row(loc_pane,xzoom_slider)
    main = pn.Column(feature_pane, seq_pane, sizing_mode='stretch_width')
    app = pn.Column(top,slider,main, sizing_mode='stretch_width',width_policy='max',margin=20)
    return app


In [85]:
genome_features_viewer('d378_attb-entry.gb')


Column(margin=20, sizing_mode='stretch_width', width_policy='max')
    [0] Row
        [0] TextInput(name='location', width=150)
        [1] IntSlider(end=500, name='zoom', start=1, step=5, value=100, width=100)
    [1] IntSlider(end=4411, name='start', step=500, value=1, width=900)
    [2] Column(sizing_mode='stretch_width')
        [0] Bokeh(Figure, height=100, margin=10)
        [1] Bokeh(Figure, height=50, margin=10)

In [9]:
def plot_sequence_mod(seq, plot_width=1000, plot_height=20, fontsize='10pt', xaxis=True, tools=""):
    """Plot a single sequence.

    Args:
        seq: sequence to plot, a string
        xaxis: display x-axis tick labels or not
        tools: which bokeh tools to display, if needed
    """

    if seq is None or len(seq)==0:
        return plotters.plot_empty('no sequence',plot_width=plot_width,plot_height=plot_height)
    text = list(seq)
    N = len(seq)
    x = np.array(range(N))+1

    colors = utils.get_sequence_colors([seq])
    source = ColumnDataSource(dict(x=x, text=text, colors=colors))
    x_range = Range1d(0,N, bounds='auto')
    p = figure(plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=(0,1),
               tools=tools, min_border=0, toolbar_location='below')
    rects = Rect(x="x", y=0,  width=1, height=2, fill_color="colors", line_color=None, fill_alpha=0.4)
    p.add_glyph(source, rects)
    if len(seq)<500:
        glyph = Text(x="x", y=0, text="text", text_align='center', text_color="black",
                     text_font="monospace", text_font_size=fontsize)
        p.add_glyph(source, glyph)
    p.grid.visible = False
    if xaxis == False:
        p.xaxis.visible = False
    else:
        if plot_height<40:
            p.plot_height = 50
        if tools != "":
            p.plot_height = 70
    p.yaxis.visible = False
    p.toolbar.logo = None
    return p